In [ ]:
import requests
import re
import urllib 
from bs4 import BeautifulSoup
import string
import pandas as pd
import numpy as np

In [ ]:
temporary_links = []
primary_links = []
product_links = []
titles = []
prices = []
marks=[]
articles=[]
categories=[]
url = 'https://www.master-instrument.ru/catalog/'
r = requests.get(url)
html_content = r.text
soup = BeautifulSoup(html_content,'html.parser')
for link in soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_="menu-item-1"):
    temporary_links.append(link.get('href'))
for link in np.unique(temporary_links)[:2]:
    url = 'https://www.master-instrument.ru' + link
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content,'html.parser')
    for link in soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_="catalog-item"):
                primary_links.append(link.get('href'))
    if soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_="catalog-item")==[]:
        primary_links.append(link)


for primary_link in primary_links:
    url = 'https://www.master-instrument.ru' + primary_link
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content,'html.parser')
    #temporary_links = []
    
    if soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_="catalog-item")!=[]:
        for link in soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_="catalog-item"):
            primary_links.append(link.get('href'))
    
        
for link in primary_links:
    url = 'https://www.master-instrument.ru' + link
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content,'html.parser')
    for link in soup.findAll('a', attrs={'href': re.compile("/catalog/")},class_="h1"):
        product_links.append(link.get('href'))
        
product_links_unique=np.unique(product_links) 

In [ ]:
for product_link in product_links_unique:
        url = 'https://www.master-instrument.ru' + product_link
        r = requests.get(url)
        html_content = r.text
        soup = BeautifulSoup(html_content,'html.parser')

        title = soup.findAll('div',class_="name")[0].text
        titles.append(title)

        if soup.findAll('div',class_="id")!=[]:
            mark = soup.findAll('div',class_="id")[0].text
        else:
            mark =[]
        marks.append(mark)

        article = str(soup.findAll('meta', attrs={'itemprop': 'brand'})[0]).split()[1][9:-1]
        articles.append(article)

        if soup.findAll('span',class_="price-label")!=[]:
            price=soup.findAll('span',class_="price-label")[0].text.split()[:-1]
        else:
            price=soup.findAll('div',class_="price-label")[0].text.split()[:-1]
            
        b=''
        for i in range(int(len(price))):
            b=str(b)+price[i]
        prices.append(b)

        category=soup.findAll('span', attrs={'itemprop': 'title','class':[]})[-1].text
        categories.append(category)

product_linkks =[ 'https://www.master-instrument.ru%s'%x for x in product_links ]

table = pd.DataFrame(list(zip(titles, marks, articles,prices, categories,product_linkks)), columns =['Наименование товара', 'Бренд', 'Артикул','Цена, руб.','Категория','Ссылка на товар']) 

table.to_excel('master-instrument.xlsx')